## 1. Import Packages and Dataset 

In [209]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
pd.set_option('display.max_columns', None)

In [294]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

def rest_days(team):
    team['date_time'] = pd.to_datetime(team['date'])
    team['rest'] = (team['date_time'] - team['date_time'].shift(1)).dt.days
    team['rest'] = team['rest'].fillna(0)
    team['rest'] = team['rest'].astype(int)
    del team['date_time']
    return team

def winrate(team):
    total = team['Wins'] + team['Losses']
    total_opp = team['Wins_opp'] + team['Losses_opp']
    team['winrate'] = team['Wins'] / total
    team['winrate_opp'] = team['Wins_opp'] / total_opp
    return team

def differential(team):
    team['differential'] = team['Total'] - team['Total_opp']
    return team

def find_team_exp_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=5, adjust=False).mean()
    return rolling

def find_team_exp_average_9(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=9, adjust=False).mean()
    return rolling

def find_team_exp_average_12(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=12, adjust=False).mean()
    return rolling

def find_team_average_15(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(15).mean()
    return rolling

def find_team_average_10(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(10).mean()
    return rolling

def find_team_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(5).mean()
    return rolling

def find_team_average_3(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(3).mean()
    return rolling

def rolling(data):
    df_rolling_3 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_3)
    df_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_5 = df_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_5)
    df_rolling_10 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_10 = df_rolling_10.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_10)
    df_rolling_15 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_15 = df_rolling_15.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_15)
    df_exp_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_5 = df_exp_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_5)
    df_exp_rolling_9 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_9 = df_exp_rolling_9.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_9)
    df_exp_rolling_12 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_12 = df_exp_rolling_12.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_12, include_groups=False)
    exp_rolling_columns_5 = [f"{col}_exp_5" for col in df_exp_rolling_5.columns]
    exp_rolling_columns_9 = [f"{col}_exp_9" for col in df_exp_rolling_9.columns]
    exp_rolling_columns_12 = [f"{col}_exp_12" for col in df_exp_rolling_12.columns]
    rolling_columns_15 = [f"{col}_15" for col in df_rolling_15.columns]
    rolling_columns_10 = [f"{col}_10" for col in df_rolling_10.columns]
    rolling_columns_5 = [f"{col}_5" for col in df_rolling_5.columns]
    rolling_columns_3 = [f"{col}_3" for col in df_rolling_3.columns]
    df_exp_rolling_12.columns = exp_rolling_columns_12
    df_exp_rolling_9.columns = exp_rolling_columns_9
    df_exp_rolling_5.columns = exp_rolling_columns_5
    df_rolling_15.columns = rolling_columns_15
    df_rolling_10.columns = rolling_columns_10
    df_rolling_5.columns = rolling_columns_5
    df_rolling_3.columns = rolling_columns_3
    df = pd.concat([data, df_rolling_3, df_rolling_5, df_rolling_10, df_rolling_15,df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12], axis=1)
    return df

def ratio(feature):
    feature_opp = 'OPP_' + str(feature)
    free = nba[feature] / nba[feature_opp]
    return free

def ratios(nba):
    regard = []
    disregard = [col for col in nba.columns if "OPP_" in col]
    for col in disregard:
        col = col[4:100]
        if col in nba.columns:
            regard.append(col)
    nba_ratio = nba[regard].apply(ratio)
    nba_ratios_columns = [f"{col}_ratio" for col in nba_ratio.columns]
    nba_ratio.columns
    return regard

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

def date_change(datetime_str):
    # Parse the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y')

    # Format the datetime object into a new string structure
    new_datetime_str = datetime_obj.strftime('%Y-%m-%d')

    return new_datetime_str

def haircut(df, date):
    df[date] = df[date].str[:10]
    return df

def convert_date_format(df):
    # Create a boolean mask to identify values in the "m/d/y" format
    mask = df['Date'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}')
    
    # Apply the conversion only to values that match the mask
    df.loc[mask, 'Date'] = nba.loc[mask, 'Date'].apply(date_change)
    return df

In [295]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/NBA_2018_2024.csv"
df = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
nba = pd.read_csv(folder_path, index_col=0)

## 2. Data Wrangling and Preprocessing for Modeling
- In this code, I have scaled the ranking data

In [296]:
# nba dataframe does not include the 2018 season
df = df[~df['season'].isin([2018])]
df = df.reset_index(drop=True)
df = haircut(df, 'date')

# rename nba columns to match df
nba = haircut(nba, 'Date')
nba = convert_date_format(nba)
nba.rename(columns={'Date': 'date_next', 'Teams':'Teams_x'}, inplace=True)

# construct winrate for team
df = winrate(df)
# construct differential points
df = differential(df)
# construct target
df = df.groupby("Teams", group_keys=False).apply(add_target)
# construct resting
df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
# games yet to play are 2
df.loc[pd.isnull(df['target']), 'target'] = 2
# convert win/loss to 1/0
df['target'] = df['target'].astype(int)

# remove metadata and target for df
removed = ['target', 'date', 'Teams_opp', 'Teams',
           'season','won', 'Wins', 'Losses', 
           'Wins_opp', 'Losses_opp']
valid_columns = df.columns[~df.columns.isin(removed)]

# scale the data
scaler = MinMaxScaler()
df[valid_columns] = scaler.fit_transform(df[valid_columns])

# construct rolling features to df
df = rolling(df).copy()
df = df.dropna()

# remove metadata for nba ranking 
removed = ['date_next', 'Teams_x']
valid_columns = nba.columns[~nba.columns.isin(removed)]

# scale the ranking data
scaler = MinMaxScaler()
nba[valid_columns] = scaler.fit_transform(nba[valid_columns])

# construct current game metadata
df['home_next'] = add_col(df, 'home')
df['team_next_opp'] = add_col(df, 'Teams_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

# merge stats from opposing teams
full = df.merge(df,
               left_on=['Teams', 'date_next'],
               right_on = ['team_next_opp', 'date_next'])

# merge stats from nba dataframe
complete = pd.merge(full, nba, on=['Teams_x', 'date_next'], how='left')
complete = complete.dropna()

# remove metadata and useless data
disregard = list(complete.columns[complete.dtypes == 'object']) 
disregard = disregard + ["home_opp_5_x","target_x","target_y", 
                         "Wins_x", "Losses_x", "Wins_opp_x", 
                         "Losses_opp_x", "season_x" , "won_x" , 
                         "home_5_x" ,"home_10_x" ,"season_5_x", 
                         "season_10_x" , "Wins_y" , "Losses_y" , 
                         "Wins_opp_y" , "Losses_opp_y" , "season_y" , 
                         "won_y" ,"home_5_y", "home_10_y","season_5_y", 
                         "season_10_y","home_opp_5_y", "home_opp_10_y"]
regard = complete.columns[~complete.columns.isin(disregard)]

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_11906/2391197117.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Teams", group_keys=False).apply(add_target)
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_11906/2391197117.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
/var/folders/99/zv3w

In [297]:
# want to exclude January 2024 - March 2024 data from dataframe to use for out of sample testing
complete = complete[~complete['date_next'].str.contains('2024-03')|complete['date_next'].str.contains('2024-02')|complete['date_next'].str.contains('2024-01')]
complete = complete.reset_index(drop=True)

In [298]:
# save processed dataframe
# save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/processed_data/'
# file = 'processed_2019_2024.csv'
# file_name = save_path + file
# complete.to_csv(file_name)

## 3. Modeling without Split

In [183]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

# can also try direction backward
sfs = SequentialFeatureSelector(rr, n_features_to_select = 70, direction='forward', cv=split)

In [184]:
sfs.fit(complete[regard], complete['target_x'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=70)

In [185]:
predictors = list(regard[sfs.get_support()])

In [186]:
predictors

['usg%_x',
 'pfmaxes_x',
 'usg%_opp_x',
 'winrate_x',
 'winrate_opp_x',
 'usg%_3_x',
 'fg%maxes_3_x',
 'usg%_opp_3_x',
 'drbmaxes_opp_3_x',
 'tovmaxes_opp_3_x',
 'winrate_3_x',
 'ast%_5_x',
 'usg%_5_x',
 'tov%_opp_5_x',
 'usg%_opp_5_x',
 'ptsmaxes_opp_5_x',
 'TOV%_opp_5_x',
 'usg%_10_x',
 '3par_opp_10_x',
 'usg%_opp_10_x',
 'fg%_15_x',
 'usg%_15_x',
 'usg%_opp_15_x',
 'stl%maxes_opp_15_x',
 'ast%_exp_5_x',
 'usg%_exp_5_x',
 'usg%_opp_exp_5_x',
 'usg%_exp_9_x',
 'usg%_opp_exp_9_x',
 'ts%maxes_opp_exp_9_x',
 'usg%_exp_12_x',
 'drbmaxes_exp_12_x',
 'usg%_opp_exp_12_x',
 'winrate_exp_12_x',
 'usg%_y',
 'usg%_opp_y',
 '3pmaxes_opp_y',
 'ftrmaxes_opp_y',
 'drb%maxes_opp_y',
 'usg%_3_y',
 'usg%_opp_3_y',
 'usg%_5_y',
 'usg%_opp_5_y',
 'trbmaxes_opp_5_y',
 'usg%_10_y',
 'usg%_opp_10_y',
 'winrate_10_y',
 'usg%_15_y',
 'usg%_opp_15_y',
 'usg%_exp_5_y',
 'ast%_opp_exp_5_y',
 'usg%_opp_exp_5_y',
 'usg%_exp_9_y',
 'usg%_opp_exp_9_y',
 'winrate_exp_9_y',
 'trb%_exp_12_y',
 'usg%_exp_12_y',
 'usg%_o

In [301]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score,f1_score
import pickle 

def backtest(data, model, predictors, true):
    X = data[predictors]
    y = data['target_x']
    
    test_size = len(y) // 4
    
    best_accuracy = 0
    best_f1 = 0
    best_model = None
    fold = 0
    
    tscv = TimeSeriesSplit(n_splits=3, test_size=test_size)
    accuracy_scores = []
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, predictions)
        f1 = f1_score(y_test, predictions)
        accuracy_scores.append(accuracy)
    
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_f1 = f1
            best_model = model
            fold = len(accuracy_scores)

        print(f'Accuracy for fold {len(accuracy_scores)}: {accuracy}')
        print(f'F1 for fold {len(accuracy_scores)}: {f1}')
        print("\n")

    if (True):
        count = len(predictors)
        accuracy = round(best_accuracy, 3) * 100
        save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
        file_path = save_path + f'ridge_classifier_{count}_predictors_{accuracy}%_2019_2024.pkl'
        with open(file_path, 'wb') as f:
            pickle.dump(best_model, f)
        
        save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
        file_path = save_path + f'predictors_ridge_classifier_{count}_predictors_{accuracy}%_2019_2024.txt'
        with open(file_path, 'w') as f:
            for predictor in predictors:
                f.write(f'{predictor},')
    
        print('-----------------------------------Saved the best model into directory-----------------------------------')
        print(f'Best accuracy: {best_accuracy}')
        print(f'Best f1 score: {best_f1}')

In [302]:
backtest(complete, rr, predictors, False)

Accuracy for fold 1: 0.7803001154290111
F1 for fold 1: 0.7801309202926454


Accuracy for fold 2: 0.8495575221238938
F1 for fold 2: 0.8495575221238938


Accuracy for fold 3: 0.8680261639091958
F1 for fold 3: 0.8667961165048543


-----------------------------------Saved the best model into directory-----------------------------------
Best accuracy: 0.8680261639091958
Best f1 score: 0.8667961165048543


## 4. Modeling with Split
We noticed that we are training a model with the same data twice over. We are going to try to apply a list comprehension to form a dataset of home teams in x and away teams in y. 

In [206]:
# complete is dataframe with both ranking and stats 
home = complete[complete['home_x'] == 1]
home = home.reset_index(drop=True)

In [207]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

# can also try direction backward
sfs = SequentialFeatureSelector(rr, n_features_to_select = 70, direction='forward', cv=split)

In [208]:
sfs.fit(home[regard], home['target_x'])

KeyboardInterrupt: 

In [ ]:
predictors = list(regard[sfs.get_support()]

In [ ]:
backtest(home, rr, predictors, False)